In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/data.csv"
df = pd.read_csv(path)
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/bert_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,-0.460592,0.072524,0.684448,-0.517376,0.123062,0.155638,0.484082,0.748201,0.604602,...,-0.451643,-0.010428,0.690342,-0.272403,0.214033,0.605862,-0.104516,-0.237995,0.344799,-0.160459
1,1,0.524083,0.849875,-0.213335,0.572916,-0.059826,-0.354075,0.243835,0.592775,-0.143739,...,1.273206,-0.125165,-0.339189,0.002869,-0.620893,-0.020557,0.779806,-0.683723,-0.720309,-0.449116
2,2,0.074979,0.041756,0.026582,-0.027790,-0.246439,-1.702710,-0.630236,1.801328,-0.945697,...,1.219780,0.474386,0.649682,-0.104247,-0.754302,-0.882654,0.392963,-1.188588,-0.049230,0.796619
3,3,-0.699477,-0.337010,-0.046207,-0.578033,0.888192,0.614672,0.528041,-0.242611,0.615708,...,-0.876815,0.215446,0.364395,-0.457493,0.707946,0.194538,-0.953793,0.600591,1.473915,-0.770428
4,4,-0.184938,-0.501136,0.201083,-0.965849,0.382337,-0.103404,-0.259140,1.220581,0.455429,...,-0.960001,0.091283,-0.054914,-0.535585,-0.191258,1.052971,0.140482,0.038958,0.150239,0.190447


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.460592,0.072524,0.684448,-0.517376,0.123062,0.155638,0.484082,0.748201,0.604602,0.238524,...,-0.451643,-0.010428,0.690342,-0.272403,0.214033,0.605862,-0.104516,-0.237995,0.344799,-0.160459
1,0.524083,0.849875,-0.213335,0.572916,-0.059826,-0.354075,0.243835,0.592775,-0.143739,0.419015,...,1.273206,-0.125165,-0.339189,0.002869,-0.620893,-0.020557,0.779806,-0.683723,-0.720309,-0.449116
2,0.074979,0.041756,0.026582,-0.027790,-0.246439,-1.702710,-0.630236,1.801328,-0.945697,0.256814,...,1.219780,0.474386,0.649682,-0.104247,-0.754302,-0.882654,0.392963,-1.188588,-0.049230,0.796619
3,-0.699477,-0.337010,-0.046207,-0.578033,0.888192,0.614672,0.528041,-0.242611,0.615708,-0.427294,...,-0.876815,0.215446,0.364395,-0.457493,0.707946,0.194538,-0.953793,0.600591,1.473915,-0.770428
4,-0.184938,-0.501136,0.201083,-0.965849,0.382337,-0.103404,-0.259140,1.220581,0.455429,0.326256,...,-0.960001,0.091283,-0.054914,-0.535585,-0.191258,1.052971,0.140482,0.038958,0.150239,0.190447


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3483598615867724


**Decision Tree**

In [17]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [18]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [19]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [20]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [21]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [22]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.385246,0.385246,0.616109,0.323134,0.284351,0.284351,0.352196,0.220151


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3600    0.1720    0.2328       157
           2     0.4581    0.7848    0.5785       237
           3     0.1860    0.0615    0.0925       130

    accuracy                         0.4218       524
   macro avg     0.3347    0.3394    0.3013       524
weighted avg     0.3612    0.4218    0.3544       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.385246,0.385246,0.616109,0.323134,0.284351,0.284351,0.352196,0.220151
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.421756,0.421756,0.361226,0.354351


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5846    0.3158    0.4101       361
           2     0.5305    0.8029    0.6388       553
           3     0.6223    0.3824    0.4737       306

    accuracy                         0.5533      1220
   macro avg     0.5791    0.5003    0.5075      1220
weighted avg     0.5695    0.5533    0.5297      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2211    0.1338    0.1667       157
           2     0.4545    0.6962    0.5500       237
           3     0.2121    0.1077    0.1429       130

    accuracy                         0.3817       524
   macro avg     0.2959    0.3126    0.2865       524
weighted avg     0.3244    0.3817    0.3341       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.542623,0.542623,0.665412,0.468415,0.416031,0.416031,0.356531,0.328412
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.391221,0.391221,0.328412,0.333321
2,Adaboost,0.553279,0.553279,0.569533,0.529727,0.381679,0.381679,0.324443,0.334138


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9816    0.8864    0.9316       361
           2     0.8695    1.0000    0.9302       553
           3     0.9922    0.8366    0.9078       306

    accuracy                         0.9254      1220
   macro avg     0.9478    0.9077    0.9232      1220
weighted avg     0.9335    0.9254    0.9250      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2963    0.1529    0.2017       157
           2     0.4397    0.7384    0.5512       237
           3     0.2000    0.0692    0.1029       130

    accuracy                         0.3969       524
   macro avg     0.3120    0.3202    0.2852       524
weighted avg     0.3373    0.3969    0.3352       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.542623,0.542623,0.665412,0.468415,0.416031,0.416031,0.356531,0.328412
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.391221,0.391221,0.328412,0.333321
2,Adaboost,0.553279,0.553279,0.569533,0.529727,0.381679,0.381679,0.324443,0.334138
3,GBM,0.925410,0.925410,0.933455,0.924989,0.396947,0.396947,0.337265,0.335239


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6066    0.7091    0.6539       361
           2     0.7529    0.5895    0.6613       553
           3     0.5863    0.6993    0.6379       306

    accuracy                         0.6525      1220
   macro avg     0.6486    0.6660    0.6510      1220
weighted avg     0.6678    0.6525    0.6532      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2299    0.2739    0.2500       157
           2     0.4056    0.3080    0.3501       237
           3     0.2420    0.2923    0.2648       130

    accuracy                         0.2939       524
   macro avg     0.2925    0.2914    0.2883       524
weighted avg     0.3124    0.2939    0.2990       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.385246,0.385246,0.616109,0.323134,0.284351,0.284351,0.352196,0.220151
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.421756,0.421756,0.361226,0.354351
2,SVM,0.652459,0.652459,0.667828,0.653209,0.293893,0.293893,0.312373,0.298957


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4615    0.4488    0.4551       361
           2     0.5508    0.7251    0.6261       553
           3     0.5106    0.2353    0.3221       306

    accuracy                         0.5205      1220
   macro avg     0.5077    0.4697    0.4678      1220
weighted avg     0.5143    0.5205    0.4992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2639    0.2420    0.2525       157
           2     0.4641    0.5992    0.5230       237
           3     0.2568    0.1462    0.1863       130

    accuracy                         0.3798       524
   macro avg     0.3282    0.3291    0.3206       524
weighted avg     0.3527    0.3798    0.3584       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.542623,0.542623,0.665412,0.468415,0.416031,0.416031,0.356531,0.328412
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.391221,0.391221,0.328412,0.333321
2,Adaboost,0.553279,0.553279,0.569533,0.529727,0.381679,0.381679,0.324443,0.334138
3,GBM,0.925410,0.925410,0.933455,0.924989,0.396947,0.396947,0.337265,0.335239
4,SVM,0.662295,0.662295,0.668176,0.654105,0.333969,0.333969,0.319917,0.325219
5,KNN,0.520492,0.520492,0.514325,0.499239,0.379771,0.379771,0.352651,0.358421


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3098    0.6925    0.4281       361
           2     0.4706    0.2893    0.3583       553
           3     0.2877    0.0686    0.1108       306

    accuracy                         0.3533      1220
   macro avg     0.3560    0.3502    0.2991      1220
weighted avg     0.3771    0.3533    0.3169      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3036    0.6943    0.4225       157
           2     0.4726    0.2911    0.3603       237
           3     0.4211    0.0615    0.1074       130

    accuracy                         0.3550       524
   macro avg     0.3991    0.3490    0.2967       524
weighted avg     0.4092    0.3550    0.3162       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.542623,0.542623,0.665412,0.468415,0.416031,0.416031,0.356531,0.328412
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.391221,0.391221,0.328412,0.333321
2,Adaboost,0.553279,0.553279,0.569533,0.529727,0.381679,0.381679,0.324443,0.334138
3,GBM,0.925410,0.925410,0.933455,0.924989,0.396947,0.396947,0.337265,0.335239
4,SVM,0.662295,0.662295,0.668176,0.654105,0.333969,0.333969,0.319917,0.325219
5,KNN,0.520492,0.520492,0.514325,0.499239,0.379771,0.379771,0.352651,0.358421
6,GNB,0.353279,0.353279,0.377128,0.316895,0.354962,0.354962,0.409184,0.316190
